In [7]:
import pandas as pd
import numpy as np
import json 
import requests
import os
import sys
import pycountry
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.data_module import read_processed_df

# Repair channel_df

In [8]:
videos_df = read_processed_df.read_video_df()
videos_df.head()

,video_id,title,published,view_count,like_count,comment_count,duration,definition,tags,default_audio_language,madeforkid,playlist_title,channelTitle,playlist_published,playlist_numvideo
0,H_8ZVRRtiIA,"#1 AI for Good Specialization [Course 1, Week ...",2023-11-17 21:49:15+00:00,1671,34.0,1.0,0 days 00:10:01,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
1,bZwkaVnhEo4,"#2 AI for Good Specialization [Course 1, Week ...",2023-07-27 16:54:21+00:00,1299,18.0,1.0,0 days 00:07:12,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
2,hkOosfWxBK4,"#3 AI for Good Specialization [Course 1, Week ...",2023-11-17 20:13:19+00:00,584,20.0,2.0,0 days 00:03:47,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
3,OIavgUGg7mc,"#4 AI for Good Specialization [Course 1, Week ...",2023-07-27 16:54:56+00:00,619,10.0,0.0,0 days 00:04:49,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
4,eZZwySXsj-g,"#5 AI for Good Specialization [Course 1, Week ...",2023-07-27 16:55:12+00:00,579,3.0,0.0,0 days 00:05:36,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0


In [9]:
channel_df = pd.DataFrame(columns=['channel_name', 'playlist_count', 'video_count', 'subscriber_count', 'view_count', 'like_count','comment_count', 'join_date', 'country', 'description', 'link'])

for channel in videos_df.channelTitle.unique():
    name = channel
    temp_df = videos_df[videos_df.channelTitle == channel]
    # Drop duplicates video id
    temp_df = temp_df.drop_duplicates(subset='video_id')
    playlist_count = temp_df.playlist_title.nunique()
    like_count = temp_df.like_count.sum()
    video_count = temp_df.video_id.nunique()
    view_count = temp_df.view_count.sum()
    comment_count = temp_df.comment_count.sum()

    channel_df = pd.concat([channel_df, pd.DataFrame([[name, playlist_count, video_count, view_count, like_count,comment_count]], columns=['channel_name', 'playlist_count', 'video_count', 'view_count', 'like_count', 'comment_count'])])



Because there are some missing information in the video_df, we need to request them from the YouTube API.

In [10]:
api_key = json.load(open('../data/external/API.json', 'r'))['Nam']
channel_id_nam = json.load(open('../data/external/channel_id_nam.json', 'r'))
channel_id_Phuc = json.load(open('../data/external/channel_id_Phuc.json', 'r'))
channel_ids = {**channel_id_nam, **channel_id_Phuc}

def get_country_name(country_code):
    try:
        country_name = pycountry.countries.get(alpha_2=country_code).name
    except:
        country_name = 'Not available'
    return country_name

def getChannelInfo(channel_id):
    url = 'https://www.googleapis.com/youtube/v3/channels?part=snippet%2CcontentDetails%2Cstatistics%2CbrandingSettings&id={}&key={}'.format(channel_id, api_key)
    response = requests.get(url)
    data = response.json()
    # Extract the required information
    subscriber_count = data['items'][0]['statistics']['subscriberCount']
    join_date = data['items'][0]['snippet']['publishedAt']
    country_code = data['items'][0]['snippet'].get('country', 'Not available')  # Some channels might not have a 'country' field
    link = "https://www.youtube.com/channel/" + data['items'][0]['id']
    description = data['items'][0]['snippet']['description']
    country = get_country_name(country_code)
    return subscriber_count, join_date, country, link, description


In [11]:
for channel in channel_df.channel_name:
    channel_id = channel_ids[channel]
    subscriber_count, join_date, country, link, description = getChannelInfo(channel_id)
    channel_df.loc[channel_df.channel_name == channel, ['subscriber_count', 'join_date', 'country', 'link', 'description']] = subscriber_count, join_date, country, link,description
channel_df

,channel_name,playlist_count,video_count,subscriber_count,view_count,like_count,comment_count,join_date,country,description,link
0,DeepLearningAI,17,364,271000,16853716,193994.0,6398.0,2017-08-22T22:27:58Z,Not available,Welcome to the official DeepLearning.AI YouTub...,https://www.youtube.com/channel/UCcIXc5mJsHVYT...
0,3Blue1Brown,20,177,5750000,458811087,12643439.0,448521.0,2015-03-03T23:11:55Z,United States,My name is Grant Sanderson. Videos here cover ...,https://www.youtube.com/channel/UCYO_jab_esuFR...
0,Joma Tech,10,108,2270000,187590804,6679177.0,218210.0,2016-08-31T22:22:07Z,United States,I talk about life in Silicon Valley and at big...,https://www.youtube.com/channel/UCV0qA-eDDICsR...
0,DataCamp,25,851,155000,17927443,55430.0,1913.0,2014-03-25T14:48:01Z,United States,Welcome to DataCamp's official YouTube channel...,https://www.youtube.com/channel/UC79Gv3mYp6zKi...
0,CS Dojo,9,105,1920000,80114014,1997796.0,87334.0,2016-02-26T01:49:30Z,Canada,"Hello! My name is YK, and I usually make video...",https://www.youtube.com/channel/UCxX9wt5FWQUAA...
0,sentdex,95,1221,1280000,120676584,1362276.0,135903.0,2012-12-16T20:49:11Z,United States,"Python Programming tutorials, going further th...",https://www.youtube.com/channel/UCfzlCWGWYyIQ0...
0,StatQuest with Josh Starmer,15,192,1060000,56417057,1235579.0,82326.0,2011-05-24T01:52:48Z,United States,"Statistics, Machine Learning and Data Science ...",https://www.youtube.com/channel/UCtYLUTtgS3k1F...
0,Tech With Tim,72,784,1400000,119884321,3153040.0,125261.0,2014-04-23T01:57:10Z,Canada,"Learn programming, software engineering, machi...",https://www.youtube.com/channel/UC4JX40jDee_tI...
0,365 Data Science,18,224,304000,13603729,271195.0,7785.0,2017-08-07T15:17:05Z,Bulgaria,At 365 Data Science we make #DataScience acces...,https://www.youtube.com/channel/UCEBpSZhI1X8Wa...
0,Data Professor,30,385,172000,17006302,529963.0,26732.0,2019-08-17T15:59:56Z,Thailand,"Data Science, Machine Learning, Bioinformatics...",https://www.youtube.com/channel/UCV8e2g4IWQqK7...


In [12]:
# Save to csv
channel_df.to_csv('../data/processed/df_channels_processed.csv', index=False)